In [ ]:

# consumer.py
from kafka import KafkaConsumer, KafkaProducer
import json
from configs import kafka_config

# Налаштування Kafka
KAFKA_CONFIG = {
    'bootstrap_servers': kafka_config["bootstrap_servers"],
    'security_protocol': kafka_config["security_protocol"],
    'sasl_mechanism': kafka_config["sasl_mechanism"],
    'sasl_plain_username': kafka_config["username"],
    'sasl_plain_password': kafka_config["password"]
}

your_name = "volodymyr17"
TOPIC_SENSORS = f'building_sensors_{your_name}'
TOPIC_TEMPERATURE_ALERTS = f'temperature_alerts_{your_name}'
TOPIC_HUMIDITY_ALERTS = f'humidity_alerts_{your_name}'

# Споживач (Consumer) для обробки даних з датчиків і відправки сповіщень
def consume_and_analyze_sensor_data():
    consumer = KafkaConsumer(
        TOPIC_SENSORS,
        **KAFKA_CONFIG,
        value_deserializer=lambda v: json.loads(v.decode('utf-8'))
    )
    producer = KafkaProducer(
        **KAFKA_CONFIG,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    try:
        for message in consumer:
                    sensor_data = message.value
                    temperature = sensor_data['temperature']
                    humidity = sensor_data['humidity']

                    # Перевірка температури
                    if temperature > 40.0:
                        alert = {
                            'building_id': sensor_data['sensor_id'],
                            'alert': f'Temperature too high: {temperature} C'
                        }
                        producer.send(TOPIC_TEMPERATURE_ALERTS, value=alert)
                        print(f"Температурне сповіщення: {alert}")

                    # Перевірка вологості
                    if humidity < 20.0 or humidity > 80.0:
                        alert = {
                            'building_id': sensor_data['sensor_id'],
                            'alert': f'Humidity out of range: {humidity}%'
                        }
                        producer.send(TOPIC_HUMIDITY_ALERTS, value=alert)
                        print(f"Сповіщення про вологість: {alert}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        consumer.close()
        

if __name__ == "__main__":
    consume_and_analyze_sensor_data()

Сповіщення про вологість: {'building_id': 4, 'alert': 'Humidity out of range: 18%'}
Температурне сповіщення: {'building_id': 4, 'alert': 'Temperature too high: 42 C'}
Сповіщення про вологість: {'building_id': 2, 'alert': 'Humidity out of range: 19%'}
Температурне сповіщення: {'building_id': 2, 'alert': 'Temperature too high: 44 C'}
Температурне сповіщення: {'building_id': 2, 'alert': 'Temperature too high: 44 C'}
Сповіщення про вологість: {'building_id': 2, 'alert': 'Humidity out of range: 82%'}
Температурне сповіщення: {'building_id': 2, 'alert': 'Temperature too high: 45 C'}
Температурне сповіщення: {'building_id': 2, 'alert': 'Temperature too high: 43 C'}
Сповіщення про вологість: {'building_id': 5, 'alert': 'Humidity out of range: 18%'}
Температурне сповіщення: {'building_id': 5, 'alert': 'Temperature too high: 42 C'}
Температурне сповіщення: {'building_id': 5, 'alert': 'Temperature too high: 44 C'}
Сповіщення про вологість: {'building_id': 5, 'alert': 'Humidity out of range: 16%'}